# Aula 10 - Aprendizado de Máquina

# Exercício 01:

Na aula 10 vimos o `PosModel`, um modelo simples para prever em qual posição uma pessoa jogaria na NBA dado o seu peso e altura. Os exercícios abaixo referem-se a esse modelo.


In [23]:
class PosModel:

    def __init__(self):
        self.tG = 0
        self.tC = 0
        self.n_train = 0
        self.hits_train = 0
        self.n_test = 0
        self.hits_test = 0
        
    def getSize(self, features):
        return features[0]*features[1]
    
    def getMinSize(self, x):
        return min([self.getSize(xi) for xi in x])
    
    def getMaxSize(self, x):
        return max([self.getSize(xi) for xi in x])    

    def getNumberOfHits(self, tG, tC, x, y):
        hits = 0
        n = len(y)
        for i in np.arange(n):
            pos = 'F'
            size = self.getSize(x[i])
            if size < tG:
                pos = 'G'
            elif size > tC:
                pos = 'C'
            if(pos == y[i]):
                hits = hits + 1
        return hits

    def train(self, x_train, y_train):
        self.n_train = len(y_train)
        minvalue = self.getMinSize(x_train)
        maxvalue = self.getMaxSize(x_train)
        #grid search   
        #for tg in np.arange(60*150,90*300,100):
        for tG in np.linspace(minvalue, maxvalue, 100):
            for tC in np.linspace(tG,maxvalue,100):
                hits = self.getNumberOfHits(tG, tC, x_train, y_train)
                if hits > self.hits_train:
                    self.hits_train = hits
                    self.tG = tG
                    self.tC = tC
                    
    def test(self, x_test, y_test): 
        self.n_test = len(y_test)
        self.hits_test = self.getNumberOfHits(self.tG, self.tC, x_test, y_test)
        return self.hits_test/self.n_test

__a)__ O modelo foi avaliado usando a acurácia simples. Modifique-o para que ele também possa exibir resultados referentes a:

- Matriz de confusão;
- Precisão;
- Revocação;
- F1 *score*;
    

In [24]:
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import random

class PosModel:

    def __init__(self):
        self.tG = 0
        self.tC = 0
        self.n_train = 0
        self.hits_train = 0
        self.n_test = 0
        self.hits_test = 0
        
    def getSize(self, features):
        return features[0]*features[1]
    
    def getMinSize(self, x):
        return min([self.getSize(xi) for xi in x])
    
    def getMaxSize(self, x):
        return max([self.getSize(xi) for xi in x])    

    def getNumberOfHits(self, tG, tC, x, y):
        hits = 0
        n = len(y)
        for i in np.arange(n):
            pos = 'F'
            size = self.getSize(x[i])
            if size < tG:
                pos = 'G'
            elif size > tC:
                pos = 'C'
            if(pos == y[i]):
                hits = hits + 1
        return hits

    def train(self, x_train, y_train):
        self.n_train = len(y_train)
        minvalue = self.getMinSize(x_train)
        maxvalue = self.getMaxSize(x_train)
        #grid search   
        #for tg in np.arange(60*150,90*300,100):
        for tG in np.linspace(minvalue, maxvalue, 100):
            for tC in np.linspace(tG,maxvalue,100):
                hits = self.getNumberOfHits(tG, tC, x_train, y_train)
                if hits > self.hits_train:
                    self.hits_train = hits
                    self.tG = tG
                    self.tC = tC
                    
    def test(self, x_test, y_test): 
        self.n_test = len(y_test)
        self.hits_test = self.getNumberOfHits(self.tG, self.tC, x_test, y_test)
        return self.hits_test/self.n_test
    
    def predict(self, x):
        pred = []
        for i in x:
            pos = 'F'
            size = self.getSize(i)
            if size < self.tG:
                pos = 'G'
            elif size > self.tC:
                pos = 'C'
            pred.append(pos)
        return pred
    
    def confusionMatrix(self, y_test, y_pred): 
        return confusion_matrix(y_test, y_pred, labels=['G', 'F', 'C'])
        
    def precision(self, y_test, y_pred):
        matrix = self.confusionMatrix(y_test, y_pred)
        prec_classes = []
        for i in range(matrix.shape[0]):
            prec_classes.append(matrix[i][i] / sum(matrix[i]))
        
        return prec_classes

    def recall(self, y_test, y_pred):
        matrix = self.confusionMatrix(y_test, y_pred)
        rec_classes = []
        for i in range(matrix.shape[1]):
            rec_classes.append(matrix[i][i]/sum([matrix[j][i] for j in range(matrix.shape[0])]))
        return rec_classes
    
    def f1_score(self, y_test, y_pred):
        p = self.precision(y_test, y_pred)
        r = self.recall(y_test, y_pred)
        return [(2*x*y)/(x+y) for x, y in zip(p, r)]
    
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

def train_test_split(x, y, test_pct):
    data = zip(x, y)                              # pair corresponding values  
    train, test = split_data(data, 1 - test_pct)  # split the dataset of pairs
    x_train, y_train = zip(*train)                # magical un-zip trick
    x_test, y_test = zip(*test)
    return x_train, x_test, y_train, y_test

df = pd.read_csv("nba_ht_wt.csv")
data = df.values
ys = data[:,1]        #posicoes
xs = data[:,2:4]      #altura e peso
x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.33)
    
model = PosModel()
model.train(x_train, y_train)
print("acurácia no conjunto de treino:", model.hits_train/model.n_train)
print("acurácia no conjunto de teste:", model.test(x_test, y_test))

pred = model.predict(xs)

print('matriz de confusão: \n', model.confusionMatrix(ys, pred))
print('precisão:', model.precision(ys, pred))
print('revocação:', model.recall(ys, pred))
print('f1 score:', model.f1_score(ys, pred))

acurácia no conjunto de treino: 0.7988505747126436
acurácia no conjunto de teste: 0.802547770700637
matriz de confusão: 
 [[175  27   0]
 [ 19 190   2]
 [  0  53  39]]
precisão: [0.8663366336633663, 0.9004739336492891, 0.42391304347826086]
revocação: [0.9020618556701031, 0.7037037037037037, 0.9512195121951219]
f1 score: [0.8838383838383839, 0.7900207900207901, 0.5864661654135338]


__b)__ (**opcional**) Talvez você tenha pensado em uma maneira melhor de prever as posições dos jogadores a partir da sua altura e peso. Caso isso seja verdade, crie o seu próprio modelo e o compare com o `PosModel`.



In [17]:
# YOUR CODE HERE
raise NotImplementedError()


NotImplementedError: 

__c)__ Considerando que o seu modelo é preciso o suficiente, aplique-o sobre outros conjuntos de dados para descobrir em quais posições as pessoas jogariam na NBA, ou seja, dado um conjunto de pessoas, com as suas respectivas alturas e pesos, quantos seriam armadores, alas e pivôs? Responda a essa pergunta para os seguintes conjuntos de dados:

- [Jogadores de hockey da NHL](https://www.dropbox.com/s/mbqrji2376qz6v8/nhl_ht_wt.csv?dl=0);
- [Jogadores de futebol da Premier League (liga inglesa)](https://www.dropbox.com/s/i5ysl5lhqlgsxv7/epl_2015_ht_wt.csv?dl=0);
- [Pessoas do conjunto de dados `Davis`](https://www.dropbox.com/s/xtexorbiu23wxlo/Davis.csv?dl=0).



In [25]:
# YOUR CODE HERE
import collections
import pandas as pd
import numpy as np
import random

df = pd.read_csv("nhl_ht_wt.csv")
data = df.values
xs = data[:,3:5]      
predict = model.predict(xs)
count = collections.Counter(predict)
print('NHL:', 'G:', count['G'], 'F:', count['F'], 'C:', count['C'], '\n')

df = pd.read_csv("epl_2015_ht_wt.csv")
data = df.values
xs = data[:,5:7]      
predict = model.predict(xs)
count = collections.Counter(predict)
print('Premier League:', 'G:', count['G'], 'F:', count['F'], 'C:', count['C'], '\n')

df = pd.read_csv("Davis.csv")
data = df.values
xs = data[:,2:4]      
predict = model.predict(xs)
count = collections.Counter(predict)
print('Davis:', 'G:', count['G'], 'F:', count['F'], 'C:', count['C'])

NHL: G: 628 F: 89 C: 0 

Premier League: G: 523 F: 3 C: 0 

Davis: G: 190 F: 9 C: 1
